In [18]:
import requests

url = "http://localhost:3000/api/project"
root = "/Users/zachsharma/Downloads"
files = [
    ('dxf', open(f'{root}/[2120] Scoring Mount - Scoring Front Plate.dxf', 'rb')),
    ('dxf', open(f'{root}/[2120] Scoring Mount - Scoring Mount.dxf', 'rb'))
]

response = requests.post(url, files=files)
slug = response.json()['slug']

In [19]:
import pymongo
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.4.2")
mydb = myclient["test"]
files = mydb["userDxf.files"]
dxfs = files.find({"filename": {
    "$regex": f"^{slug}",
}})

In [ ]:
url = f"http://localhost:3000/api/project/{slug}/nest"
payload = {
    "files": [
        {"slug": name, "count": 1} for name in [doc['filename'] for doc in dxfs]
    ],
    "params": {
        "width": 400,
        "height": 560,
        "tolerance": 0.1,
        "space": 0.1
    }
}
print(payload)
headers = {"Content-Type": "application/json"}

response = requests.post(url, json=payload, headers=headers)

{'files': [{'slug': 'clever-einstein-BZKkfr-2120-scoring-mount-scoring-front-platedxf-HDgS5r', 'count': 1}, {'slug': 'clever-einstein-BZKkfr-2120-scoring-mount-scoring-mountdxf-CjrdnP', 'count': 1}], 'params': {'width': 400, 'height': 560, 'tolerance': 0.1, 'space': 0.1}}
{
  "jobId": "67dd0ac35e79f0e2757425f2",
  "slug": "nest-zL8yaU"
}


In [27]:
collection = mydb['nest_request']
import time
while True:
    nest = list(collection.find({'projectSlug': slug}))
    for doc in nest:
        with open(f"{slug}.dxf", "wb") as f:
            f.write(doc['dxfResult'])
        break
    if len(nest) != 0:
        break
    else:
        time.sleep(5)

In [40]:
import ezdxf
from geomdl import BSpline, utilities

def fix_spline(spline, msp):
    # Convert control points to list of lists (each as [x, y, z])
    ctrl_pts = [list(pt) for pt in spline.control_points]
    if not ctrl_pts:
        print(f"Spline {spline.dxf.handle} has no control points; skipping.")
        return

    # Use existing degree if available; otherwise default to 3
    degree = getattr(spline.dxf, "degree", 3)

    # Get weights if available; otherwise default to 1.0 for each control point
    try:
        weights = list(spline.dxf.weights)
        if not weights:
            raise AttributeError
    except AttributeError:
        weights = [1.0] * len(ctrl_pts)

    # Use geomdl to generate a proper knot vector and construct a BSpline curve
    curve = BSpline.Curve()
    curve.degree = degree
    curve.ctrlpts = ctrl_pts
    curve.weights = weights
    curve.knotvector = utilities.generate_knot_vector(degree, len(ctrl_pts))

    # Create a new spline entity (empty) in modelspace
    new_spline = msp.add_spline()
    # Apply the BSpline construction tool to set knots, weights, etc.
    new_spline.apply_construction_tool(curve)
    # Optionally copy attributes, for example the layer
    new_spline.dxf.layer = spline.dxf.layer

    # Delete the old spline entity
    msp.delete_entity(spline)

def main():
    input_file = "./importedDxfs/output_fixed2.dxf"      # Change to your DXF file
    output_file = "output_fixed.dxf"
    doc = ezdxf.readfile(input_file)
    msp = doc.modelspace()

    for spline in list(msp.query("SPLINE")):
        fix_spline(spline, msp)

    doc.saveas(output_file)
    print("Fixed DXF saved as", output_file)

if __name__ == '__main__':
    main()

Fixed DXF saved as output_fixed.dxf


In [59]:
chunks = mydb['userDxf.chunks']
chunks.delete_many({})
files = mydb['userDxf.files']
files.delete_many({})
nest = mydb['nest_request']
nest.delete_many({})
projects = mydb['projects']
projects.delete_many({})

DeleteResult({'n': 6, 'ok': 1.0}, acknowledged=True)

In [3]:
import pymongo
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.4.2")
db = client['jira']
tasks = db['tasks']
imported = db['imported']
tasks.insert_many([
    {
        "Material": "Polycarb",
        "Thickness": 0.25,
        "Parts": [],
        "Status": "Not Started",
    },
    {
        "Material": "Polycarb",
        "Thickness": 0.125,
        "Parts": [],
        "Status": "Not Started",
    },
    {
        "Material": "Polycarb",
        "Thickness": 0.0625,
        "Parts": [],
        "Status": "Not Started",
    },
    {
        "Material": "Aluminum",
        "Thickness": 0.25,
        "Parts": [],
        "Status": "Not Started",
    },
    {
        "Material": "Aluminum",
        "Thickness": 0.125,
        "Parts": [],
        "Status": "Not Started",
    },
    {
        "Material": "Aluminum",
        "Thickness": 0.0625,
        "Parts": [],
        "Status": "Not Started",
    }
])

InsertManyResult([ObjectId('67e04216436763982b15667f'), ObjectId('67e04216436763982b156680'), ObjectId('67e04216436763982b156681'), ObjectId('67e04216436763982b156682'), ObjectId('67e04216436763982b156683'), ObjectId('67e04216436763982b156684')], acknowledged=True)

In [1]:
import selenium
from selenium import webdriver
import selenium.webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.remote
import selenium.webdriver.remote.webelement
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pymongo
import time
from bs4 import BeautifulSoup as bs
import subprocess
import json
import requests
from threading import Thread
from autodeskAutomation.macro import login_to_jira, element_containing, options
createTicket = webdriver.Chrome(options=options)
login_to_jira(createTicket)
createTicket.get('https://jira.valor6800.com/browse/HAR-800')

# element_containing(id='customfield_10110-field', specificType='input', driver=createTicket).send_keys('CAM' + Keys.ARROW_DOWN + Keys.ENTER)

In [2]:
element_containing(id='opsbar-transitions_more', specificType='a', driver=createTicket).click()
time.sleep(1)
createTicket.find_element(By.XPATH, "//*[text()='Ready' and contains(@class, 'workflow-cell')]/../..").click()
# element_containing(id='opsbar-transitions_more', specificType='a', driver=createTicket).send_keys(Keys.RETURN)

In [12]:
element_containing(id='opsbar-transitions_more', specificType='a', driver=createTicket).send_keys(Keys.DOWN)